In [1]:
import csv
import networkx as nx
import numpy as np
from random import randint
from random import random
import pandas as pd
from unidecode import unidecode
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer


from scipy.sparse import identity, diags, csr_matrix
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.metrics import log_loss, accuracy_score


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
def text_to_list(text):
    return unidecode(text).split(',')

In [28]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [3]:
def new_normalize_adjacency(A):
    n = A.shape[0] 
    A = A + identity(n)
    return A

In [4]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [5]:
class MessagePassing(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MessagePassing, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x, adj):
        
        x = self.cf(x)
        out = torch.mm(adj, x)
        
        return out

In [6]:
class GNN(nn.Module):
    def __init__(self, n_feat, n_hidden, n_class, dropout):
        super(GNN, self).__init__()
#         self.fc11 = nn.Linear(n_feat, round(n_feat/10))
#         self.fc12 = nn.Linear(round(n_feat/10), round(n_feat/100))
#         self.fc13 = nn.Linear(round(n_feat/100), round(n_feat/200))
#         self.fc14 = nn.Linear(round(n_feat/200), n_hidden)
        self.fc11 = nn.Linear(n_feat, n_hidden)
        self.fc12 = nn.Linear(n_hidden, n_hidden)
        self.fc13 = nn.Linear(n_hidden, n_hidden)
        self.fc14 = nn.Linear(n_hidden, n_hidden)
                              
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, n_class)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x_in, adj, pairs):
        
        # Authors embedding using 4 levels MLP to densify representation from sparse 147950 features to n_hidden embedded features
        h11 = self.fc11(x_in)
        z11 = self.relu(torch.mm(adj, h11)) # remove the multiplication with adj each time ?
#         h12 = self.fc12(z11)
#         z12 = self.relu(torch.mm(adj, h12))
#         h13 = self.fc13(z12)
#         z13 = self.relu(torch.mm(adj, h13))
#         h14 = self.fc13(z13)
#         z14 = self.relu(torch.mm(adj, h14))
        
        z1 = self.dropout(z11)
        #print('h1.shape=', z1.shape, ' adj.shape=', adj.shape)
        h2 = self.fc2(z1)
        
        z2 = self.relu(torch.mm(adj, h2))
        
        x = z2[pairs[0,:],:] - z2[pairs[1,:],:]
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return F.log_softmax(x, dim=1)

In [7]:
G = nx.read_edgelist('../input_data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
edges = list(G.edges())

val_edges = list()
G_train = G

for edge in edges:
    if random() < 0.1:
        val_edges.append(edge)

# We remove the val edges from the graph G
for edge in val_edges:
    G_train.remove_edge(edge[0], edge[1])

n = G_train.number_of_nodes()
m = G_train.number_of_edges()
train_edges = list(G_train.edges())
    
print('Number of nodes of training set:', n)
print('Number of edges of training set:', m)

y_val = [1]*len(val_edges)

n_val_edges = len(val_edges)

# Create random pairs of nodes
for i in range(n_val_edges):
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    (n1, n2) = (min(n1, n2), max(n1, n2))
    val_edges.append((n1, n2))
    
# Remove from val_edges edges that exist in both train and val

for edge in list(set(val_edges) & set(train_edges)):
    val_edges.remove(edge)
    
n_val_edges = len(val_edges) - len(y_val) #because we removed from val_edges edges that exist in both
y_val.extend([0]*n_val_edges)

Number of nodes of training set: 138499
Number of edges of training set: 982880


In [27]:
edges

[(0, 1),
 (0, 2),
 (1, 3),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (2, 25),
 (2, 26),
 (2, 27),
 (2, 28),
 (2, 29),
 (2, 30),
 (2, 31),
 (2, 32),
 (2, 33),
 (2, 34),
 (2, 35),
 (2, 36),
 (2, 37),
 (2, 38),
 (2, 39),
 (2, 40),
 (2, 41),
 (2, 42),
 (2, 43),
 (2, 44),
 (2, 45),
 (2, 46),
 (2, 48),
 (2, 49),
 (2, 50),
 (2, 51),
 (2, 52),
 (2, 53),
 (2, 54),
 (2, 55),
 (2, 56),
 (2, 57),
 (2, 58),
 (2, 59),
 (2, 60),
 (2, 61),
 (2, 62),
 (2, 63),
 (3, 48051),
 (3, 79497),
 (3, 101494),
 (3, 111705),
 (3, 62202),
 (3, 124845),
 (3, 124846),
 (3, 107826),
 (3, 111339),
 (3, 110133),
 (5, 34604),
 (5, 17314),
 (5, 47062),
 (5, 17446),
 (5, 71762),
 (5, 60676),
 (5, 48338),
 (5, 55141),
 (5, 58351),
 (5, 58361),
 (5, 46368),
 (5, 6),
 (5, 92524),
 (5, 68395),
 (5, 45676),
 (5, 92944),
 (5, 68327),
 (5, 90583),
 (5, 68398),
 (5, 52644),
 (5, 89340),
 (6, 16),
 

In [38]:
same_authors_edges = []
nb_same_author = 0
for i, edge in enumerate(edges):
    if len(intersection(authors['authors'][edge[0]], authors['authors'][edge[1]]))>0:
        same_authors_edges.append((edge[0], edge[1], 1))
        nb_same_author += 1
    else:
        same_authors_edges.append((edge[0], edge[1], 0))
print(nb_same_author, len(edges))

117041 1091955


In [36]:
authors[authors['paper_id'] == 58351]

,paper_id,1,authors
58351,58351,--,"[S. Chen, X. Hong, C. J. Harris]"


In [37]:
authors[authors['paper_id'] == 5]

,paper_id,1,authors
5,5,--,"[S. Chen, X. Hong, B. L. Luk, C. J. Harris]"


In [10]:
# features initializaed with a sparce representation of authors of the papers

authors = pd.read_csv('../input_data/authors.txt', sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(authors['authors']),columns=mlb.classes_, index=authors.index)
features_np = df.values

features_np = np.random.randn(G_train.number_of_nodes(), 32) # Generates node features randomly
print(type(features_np))
features_np.shape

<class 'numpy.ndarray'>


(138499, 32)

In [ ]:
# from gensim.models import Word2Vec

# model = Word2Vec(vector_size=1000, window=5, min_count=0, sg=1, workers=8)
# model.build_vocab(features_np)
# model.train(features_np, total_examples=model.corpus_count, epochs=5) 
# model.wv['32098']

In [ ]:
adj = nx.adjacency_matrix(G_train)# Obtains the adjacency matrix of the training graph
print(type(adj))
print(adj.shape)
adj = new_normalize_adjacency(adj) # Normalizes the adjacency matrix only by adding ones to diag
print(type(adj), '\n')

indices = np.array(adj.nonzero())

# Create class labels
y = np.zeros(2*len(indices[0]))
y[:len(indices[0])] = 1 # Concatenated ones for edges indices and zeros for random indices.

# Transforms the numpy matrices/vectors to torch tensors.
print(type(features_np))
#features_np = csr_matrix(features_np)
#print(type(features_np))
#features = sparse_mx_to_torch_sparse_tensor(features_np).to(device)
#print(type(features_np))
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
indices = torch.LongTensor(indices).to(device)


In [ ]:
# Hyperparameters
epochs = 20
n_hidden = 128
dropout_rate = 0.2
n_class = 2
n_features = features.shape[1]

# Creates the model and specifies the optimizer
model = GNN(n_features, n_hidden, n_class, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
print(features.shape)
print(adj.shape)

In [ ]:
import time

# Train model
model.train()
start_time = time.time()
for epoch in range(epochs):
    t = time.time()
    optimizer.zero_grad()
    rand_indices = torch.randint(0, features.shape[0], (indices.shape[0],indices.shape[1]), device=adj.device)# We take random indices each time we run an epoch
    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices.   
    
    output = model(features, adj, pairs) # we run the model that gives the output.
    loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
    #print(type(loss_train), '\n', loss_train.shape)
    acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
    loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
    optimizer.step() # Performs a single optimization step (parameter update).
    
    if epoch % 5 == 0:
        print('Epoch: {:03d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'time: {:.4f}s'.format(time.time() - t),
             'total_time: {}min'.format(round((time.time() - start_time)/60)))

print("Optimization Finished in {} min!".format(round((time.time() - start_time)/60)))
print()

In [ ]:
# Validation on val subset then calculate loss between prediction (y_pred) and valid y (y_val)
node_pairs = np.array(np.transpose(val_edges))
pairs = torch.LongTensor(node_pairs).to(device)
pred_output = model(features, adj, pairs)
y_pred = torch.exp(pred_output)
y_pred = y_pred.detach().cpu().numpy()
print('Log loss:', log_loss(y_val, y_pred))

In [ ]:
print(len(pairs[0]), len(indices[0]), adj.shape, len(y))

In [ ]:
len(pairs[0])

In [ ]:
#y_val
y_pred

In [ ]:
# features initializaed randomly because not yet ready
features_np = np.random.randn(G_train.number_of_nodes(), 32) # Generates node features randomly
features_np.shape


In [ ]:
print(indices.shape)
print(min(features_np[0]))

In [ ]:
G_train.number_of_edges()*2 # = 1965036 